In [1]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from numpy import array, savetxt
from skimage.feature import local_binary_pattern
import numpy as np
import pandas as pd
import PIL
import glob
from PIL import Image
import cv2
import sys
import os
import csv
from os import listdir
import matplotlib.pyplot as plt

def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center: #comparison function
            new_value = 1
    except:
        pass
    return new_value

def lbp_calculated_pixel(img, x, y):
      
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y-1))     # top_left
    val_ar.append(get_pixel(img, center, x-1, y))       # top
    val_ar.append(get_pixel(img, center, x-1, y+1))     # top_right
    val_ar.append(get_pixel(img, center, x, y+1))       # right
    val_ar.append(get_pixel(img, center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel(img, center, x+1, y))       # bottom
    val_ar.append(get_pixel(img, center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel(img, center, x, y-1))       # left
    
    power_val = [128,64,32,16,8,4,2,1]
    val = 0
    for i in range(len(val_ar)):
        val += (int)(val_ar[i] * power_val[i])

    return val 
    
def lbp(img,height,width):
    img2=np.pad(img,pad_width=1) #padding the initial matrix
    height=img2.shape[0]
    width=img2.shape[1]
    img_lbp=np.zeros((height-2, width-2), dtype=int)

    for i in range(0, height-2):
        for j in range(0, width-2):
            img_lbp[i][j] = lbp_calculated_pixel(img2, i+1, j+1) #calling the necessary functions for lbp calculation and removing the padding
                
    
    return img_lbp

def multiblock_lbp(img):
    radius = 3
    n_points = 8 * radius
    lbp = local_binary_pattern(img, n_points, radius, method='uniform')
    mb_lbp = np.zeros((8, 8), dtype=np.uint8)
    for i in range(8):
        for j in range(8):
            block = lbp[i*8:i*8+8, j*8:j*8+8]
            hist, _ = np.histogram(block, bins=np.arange(0, n_points+3), range=(0, n_points+2))
            mb_lbp[i, j] = np.argmax(hist)
    return mb_lbp  
    
def labels_pixels(size,x):
    rows=[]
    f = open(f"/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/{x}.csv", "w")
    f.truncate()
    f.close()
    
    with open(f'/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/{x}.csv', 'a') as f:
        f = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        if x=='train':
            rows.append("Labels")
        for i in range(size):
            rows.append(f'pixels{i}')   
        f.writerow(rows)  
        
    if(x=='train'):
        print("Labels for training done")
    else:
        print("Labels for testing done")
        
def createFileList(myDir, format='.png'):
    fileList = []
    print(myDir)
    for root, dirs, files in os.walk(myDir, topdown=False):
        for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
    return fileList

def train():
    # load the original image
    myFileList = createFileList('/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/trainingcnn/train')
    labels_pixels(64,'train')
    y=0
    features = []
    for file in myFileList:
        img_file = cv2.imread(file,0)
        img_lbp=multiblock_lbp(img_file)
        img_lbp = list(img_lbp.flatten())
        #print(len(img_lbp))
        path=os.path.basename(os.path.dirname(file))
        if path=='happy':
            img_lbp.insert(0,0)
        if path=='sad':
            img_lbp.insert(0,1)
        if path=='fearful':
            img_lbp.insert(0,2)
        if path=='neutral':
            img_lbp.insert(0,3)
        if path=='angry':
            img_lbp.insert(0,4)
        if path=='disgusted':
            img_lbp.insert(0,5)
        if path=='surprised':
            img_lbp.insert(0,6)
        
        with open("/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/train.csv", 'a') as f:
            writer = csv.writer(f)
            writer.writerow(img_lbp)

    file_name='/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/train.csv'      
    df = pd.read_csv(file_name) # avoid header=None. 
    shuffled_df = df.sample(frac=1)
    shuffled_df.to_csv(file_name, index=False)

    print("Training complete")

def test():
    myFileList = createFileList('/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/trainingcnn/test')
    labels_pixels(64,'test')
    for file in myFileList:
        img_file = cv2.imread(file,0)
        img_lbp=multiblock_lbp(img_file)

        img_lbp = list(img_lbp.flatten())
        path=os.path.basename(os.path.dirname(file))
        
        with open("/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/test.csv", 'a') as f:
            writer = csv.writer(f)
            writer.writerow(img_lbp)
        
    file_name='/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/test.csv'      
    df = pd.read_csv(file_name) # avoid header=None. 
    shuffled_df = df.sample(frac=1)
    shuffled_df.to_csv(file_name, index=False)

    print("Testing complete")
    

train()
test()

2023-04-26 12:11:15.161683: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/trainingcnn/train
Labels for training done
Training complete
/Users/khushm/Documents/GitHub/MINIPROJECT_CNNLBP/trainingcnn/test
Labels for testing done
Testing complete
